In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Since Alaska Airlines and Virgin America are in the process of merging I thought perhaps I could take this opportunity to examine the relative makeups of their fleets. Alaska Airlines has (I believe) in the past subscribed to the Southwest Airlines early model of using only one type of aircraft in order to greatly reduce maintenance costs. This was a key driver of Southwest Airline's early competitiveness (although their organizational culture also often receives, and apparently deserves, much of the credit for their performance.

If my first assumption is true, then it will be interesting to see if Virgin America has a fleet of similar makeup to the one employed currently by Alaska Airlines. If not that means a significant departure from their current business model and a possible long-term loss of organizational efficiency. In general terms, it seems that growth is actually a harbinger of decline for american airlines. It will be interesting to see if that is a possibility in this case.

I take some small interest in this fact because I live in  Alaska and typically travel Outside (what Alaskans call the rest of the United States) on Alaska Airlines flights.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('../input/Fleet Data.csv')
print(data.info())

Looks like quite a bit of null entries, but that may  not be an issue. This data apparently covers both past and current fleets (and probably also some airlines that no longer exist) so many fields should quite rightly be blank. It will be interesting to note the relative ages of Virgin and Alaskan aircraft later in this analysis.

In [ ]:
print(data.head())

In [ ]:
alaska = data[data['Parent Airline'] == 'Alaska Airlines']
virgin = data[data['Parent Airline'] == 'Virgin America']
alaska_current = alaska[alaska['Current'] > 0]
virgin_current = virgin[virgin['Current'] > 0]
plt.subplot(1,2,1)
ak_barplot = sns.barplot(x=alaska_current['Aircraft Type'], y=alaska_current['Current'], data=alaska_current)
plt.setp(ak_barplot.get_xticklabels(), rotation=45)
_ = plt.xlabel('Aircraft Type')
_ = plt.ylabel('Currently In Service')
_ = plt.title('Alaska Airlines Fleet Composition')
plt.subplot(1,2,2)
vg_barplot = sns.barplot(x=virgin_current['Aircraft Type'], y=virgin_current['Current'], data=virgin_current)
plt.setp(vg_barplot.get_xticklabels(), rotation=45)
_ = plt.xlabel('Aircraft Type')
_ = plt.ylabel('Currently in Service')
_ = plt.title('Virgin America Fleet Composition')
plt.tight_layout()
plt.show()

I have to say I'm a little surprised to see that the two airlines actually share NONE of their aircraft types. I am intensely curious to know how Alaska's executives plan to integrate maintenance of the two carriers. Unfortunately I can't answer that question with this dataset. What I CAN do is look at the aircraft similiarities between Alaska and Frontier, another carrier that Alaska acquired some time ago. If there is a disparity in aircraft type there as well then I can at least assume that Alaska has had experience with this issue before and is therefore (hopefully) competent to deal with again (albeit on a larger scale this time).

In [ ]:
frontier_legacy = data[data['Airline'] == 'Frontier Airlines']
alaska_legacy = alaska[alaska['Historic'] > 0]
plt.subplot(1,2,1)
barplot = sns.barplot(x=frontier_legacy['Aircraft Type'], y=frontier_legacy['Current'], data=frontier_legacy)
plt.setp(barplot.get_xticklabels(), rotation=45)
_ = plt.xlabel('Aircraft Type')
_ = plt.ylabel('Formerly in Service')
_ = plt.title('Legacy Frontier Fleet')
plt.subplot(1, 2, 2)
barplot = sns.barplot(x=alaska_legacy['Aircraft Type'], y=alaska_legacy['Historic'], data=alaska_legacy)
plt.setp(barplot.get_xticklabels(), rotation=45)
_ = plt.xlabel('Aircraft Type')
_ = plt.ylabel('Formerly in Service')
_ = plt.title('Legacy Alaska Fleet')
plt.tight_layout()
plt.show()

Well, this is actually quite encouraging. Looks like Frontier, before it was absorbed by Alaska, actually had a fleet very similar in makeup to the one currently used by Virgin. Since none of these aircraft are currently in Alaska's inventory it seems likely that Alaska phased them out. I would bet money they were replaced with Boeing 737s, which comprise the bulk of Alaska's current fleet. 

I plotted Alaska's historic fleet as well just out of curiosity to see if the legacy Frontier aircraft show up. They don't (obviously). Not sure what the significance of that is. Did they replace all Frontier aircraft immediately upon acquisition of Frontier? That seems unlikely, just because it is so expensive, but perhaps they did so to mitigate long-term maintenance costs. I don't think this dataset will answer that question and I won't look into it any further.

The important thing here is that whatever the interoperability/phasing/maintenance issues that Alaska may encounter with Virgin's fleet, it is nothing that they have not encountered before.

Just out of curiosity, I wonder what the relative values of these fleets are? Specifically, how much materiel (aircraft-wise) is Alaska actually gaining in this acquisition?

In [ ]:
aircraft = alaska_current.groupby('Aircraft Type').sum()
current_fleet = aircraft['Current']
cost = alaska_current['Unit Cost'].str.strip('$')
cost = pd.to_numeric(cost)
cost.index = current_fleet.index
ak_cost_per_unit = current_fleet * cost
ak_total_investment = ak_cost_per_unit.sum()
v_aircraft = virgin_current.groupby('Aircraft Type').sum()
v_current_fleet = v_aircraft['Current']
v_cost = virgin_current['Unit Cost'].str.strip('$')
v_cost = pd.to_numeric(v_cost)
v_cost.index = v_current_fleet.index
v_cost_per_unit = v_current_fleet * v_cost
v_total_investment = v_cost_per_unit.sum()
print(ak_cost_per_unit)
print(ak_total_investment)
print(v_cost_per_unit)
print(v_total_investment)

Well, there you go. The Alaska fleet is valued at $13.856 billion and Virgin is valued at about $6.1 billion.

Perhaps a couple of fun graphics are in order before we put this one to bed. Chiefly I am interested in showing how dominant the Boeing 737 is as a component of the Alaska fleet. It is obvious from the numbers but I'd like to display it graphically.

In [ ]:
ak_labels = ak_cost_per_unit.index
v_labels = v_cost_per_unit.index
plt.subplot(4,1,1)
_ = plt.pie(ak_cost_per_unit, explode=(.2,0,0,0), labels=ak_labels, autopct='%1.1f%%', shadow=True, startangle=90)
_ = plt.title('Aircraft Proportion of Total Fleet Value (Alaska Airlines)')
plt.subplot(4,1,2)
_ = plt.pie(current_fleet, explode=(.2,0,0,0), labels=ak_labels, autopct='%1.1f%%', shadow=True, startangle=90)
_ = plt.title('Number of Aircraft in Proportion to Total Fleet (Alaska Airlines)')
plt.subplot(4,1,3)
_ = plt.pie(v_cost_per_unit, explode=(.2,0), labels=v_labels, autopct='%1.1f%%', shadow=True, startangle=90)
_ = plt.title('Aircraft Proportion of Total Fleet Value (Virgin America)')
plt.subplot(4,1,4)
_ = plt.pie(v_current_fleet, explode=(.2,0), labels=v_labels, autopct='%1.1f%%', shadow=True, startangle=90)
_ = plt.title('Number of Aircraft in Proportion to Total Fleet (Virgin America)')
plt.tight_layout()
plt.show()

Some modestly interesting observations about the relative value of Boeing 737s are obvious here. It's interesting to note too that the two Airbus aircraft are roughly the same price. Virgin America then is subscribing to the Southwest model as well, sticking with a single line of aircraft to reduce maintenance costs and reduce the complexity of route planning.

Well, that's all I was looking to discover from this particular dataset. It's been a good exercise in Dataframe/Series manipulation and use of matplotlib for a definite Python/data science novice.